# Property Package Introduction

The properties of the solvent (usually water) in WaterTAP models are defined by the [property models](https://watertap.readthedocs.io/en/stable/technical_reference/property_models/index.html). 

In unit models, properties exist as sub-models of the main unit model on what are called _state blocks_. 

The creation of state blocks is done automatically as part of the build for any unit model.

This tutorial will cover the basics of interacting with property packages and state blocks in WaterTAP. We will do this via creation of a feed block with the `Feed` model. This model is intended to be the starting point for any WaterTAP flowsheet.

### Required imports from Pyomo, IDAES, and WaterTAP

We are importing the `Feed` unit model from IDAES and will be using the [WaterTAP seawater property model](https://watertap.readthedocs.io/en/stable/technical_reference/property_models/seawater.html) as the property package. This package implements property relationships for seawater as provided in [Sharqawy et al. (2010)](https://doi.org/10.5004/dwt.2010.1079) and [Nayar et al. (2016)](https://doi.org/10.5004/dwt.2010.1079).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pyomo.environ import ConcreteModel, Constraint, value, units as pyunits

from idaes.core import FlowsheetBlock
from idaes.models.unit_models import Feed
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
from idaes.core.util.model_statistics import degrees_of_freedom

from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock
from watertap.core.solvers import get_solver

### Create model, flowsheet, and property model

Like any WaterTAP model, we first create the model, flowsheet, and property model. 

<img src="img/step1.png" alt="Alternative text" />

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

### Add `Feed` model to flowsheet

Next, we add the feed model to the flowsheet and pass the property model we created previously via the `property_package` argument. 

As soon as this cell is run, our `feed` model is built. As part of the build process, a _state block_ is created. Despite being termed a "property model", the `m.fs.properties` block contains no properties on it. Instead, it contains all the information needed to create properties on _state blocks_.

For the `Feed` model, the _state block_ is called `properties` and is indexed to `[0]`. 

<img src="img/step2.png" alt="Alternative text" />

Like many other components, we use the `display()` method to see all the variables, constraints, and objectives on the state block.

In this case, we observe only the _state variables_. These are the specific variables used by the property model to define the state of the stream. 

For the seawater property model (and many other WaterTAP property models), the state variables are:
- pressure
- temperature
- mass flow rate of components 

For the seawater property model, the components are `H2O` and `TDS`, but other property models allow adding any number of components.

In [ ]:
m.fs.feed = Feed(property_package=m.fs.properties)
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")
m.fs.feed.properties[0].display()

### Creating other properties

Despite the number of other properties available in the seawater property model, we only see the _state variables_ here. Why?

Outside of the state variables, other property variables are only created on state blocks on an as-needed basis. That is, the model will not create these other properties unless they are needed by the model.

To create additional properties, you only need to try to access them (or, "touch" them).

Let's say our model will need the mass concentration of components, named `conc_mass_phase_comp` in our property model. Simply adding a line referencing this (as yet uncreated) property will add this variable to the state block.


In [ ]:
# Touch conc_mass_phase_comp to create it
m.fs.feed.properties[0].conc_mass_phase_comp
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")
m.fs.feed.properties[0].display()

<img src="img/step3.png" alt="Alternative text" />

### Creating any property will also create the properties that define it

Notice that our state block now includes not only `conc_mass_phase_comp`, but also `dens_mass_phase`, `mass_frac_phase_comp`, and `dens_mass_solvent`. Why?

When we created `conc_mass_phase_comp`, we also created the constraints that define `conc_mass_phase_comp`, which is:

`conc_mass_phase_comp = mass_frac_phase_comp * dens_mass_phase`

This likewise created the constraints that define `mass_frac_phase_comp`, and `dens_mass_phase`. 

`mass_frac_phase_comp` is a function of `flow_mass_phase_comp` (our state variable), so that didn't create any additional variables.

But because `dens_mass_phase` is a function of `dens_mass_solvent`, we also see `dens_mass_solvent` on our state block.
    

### Setting the state variables

For all the properties we have created to be constrained, we only need to fix the values for the state variables.

<img src="img/step4.png" alt="Alternative text" />

In [ ]:
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)
m.fs.feed.properties[0].temperature.fix(273 + 25)
m.fs.feed.properties[0].pressure.fix(101325)
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")

### Initialize and solve to determine the concentration

With the state variables fixed, we can initialize and solve the model. This will determine the values for all the variables on our state block.

In [ ]:
m.fs.feed.initialize()

solver = get_solver()
results = solver.solve(m)
print(f"Solve termination {results.solver.termination_condition}")
print(
    f"Concentration of TDS: {value(m.fs.feed.properties[0].conc_mass_phase_comp['Liq', 'TDS']):.2f} g/L"
)

### Repeat with osmotic pressure

To further demonstrate this concept, we will repeat the above steps except with the osmotic pressure property `pressure_osm_phase`, which is used in the reverse osmosis model.

In [ ]:
# Create model, flowsheet, and property package
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

# Add feed model
m.fs.feed = Feed(property_package=m.fs.properties)

# Touch pressure_osm_phase to create it
m.fs.feed.properties[0].pressure_osm_phase

# Fix state variables
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)
m.fs.feed.properties[0].temperature.fix(273 + 25)
m.fs.feed.properties[0].pressure.fix(101325)
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")

m.fs.feed.initialize()
results = solver.solve(m)

print(f"Solve termination {results.solver.termination_condition}")
print(
    f"Osmotic Pressure (Pa): {value(m.fs.feed.properties[0].pressure_osm_phase['Liq']):.2f} Pa"
)
print(
    f"Osmotic Pressure (bar): {value(pyunits.convert(m.fs.feed.properties[0].pressure_osm_phase['Liq'], to_units=pyunits.bar)):.2f} bar"
)
# m.fs.feed.properties[0].display()

### Solve for other variables using a specified osmotic pressure

Rather than fixing the state variables to determine other variables, we can also do the inverse. For example, here we determine the concentration for a given osmotic pressure.

In [ ]:
# Touch conc_mass_phase_comp for reporting purposes
m.fs.feed.properties[0].conc_mass_phase_comp
# Fix osmotic pressure to desired value
m.fs.feed.properties[0].pressure_osm_phase["Liq"].fix(12 * pyunits.bar)
# Unfix the mass flow of TDS
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].unfix()
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")

results = solver.solve(m)
print(f"Solve termination {results.solver.termination_condition}")
print(
    f"Concentration of TDS: {value(m.fs.feed.properties[0].conc_mass_phase_comp['Liq', 'TDS']):.4f} g/L"
)
# m.fs.feed.properties[0].display()

### Compare seawater and NaCl property packages

Compare osmotic pressure for low salinity (5-70 g/L)

In [ ]:
# =======================================================================================
# Build the model for seawater property package
def build_seawater_prop_model():
    m1 = ConcreteModel()
    m1.fs = FlowsheetBlock(dynamic=False)
    m1.fs.properties = SeawaterParameterBlock()
    m1.fs.feed = Feed(property_package=m1.fs.properties)
    m1.fs.feed.properties[0].pressure_osm_phase
    m1.fs.feed.properties[0].conc_mass_phase_comp
    m1.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
    m1.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)
    m1.fs.feed.properties[0].temperature.fix(273 + 25)
    m1.fs.feed.properties[0].pressure.fix(101325)

    print(f"Degrees of freedom: {degrees_of_freedom(m1)}\n")

    # Initialize and solve for the initial conditions
    m1.fs.feed.initialize()
    results = solver.solve(m1)
    print(f"Solve termination {results.solver.termination_condition}")
    # Unfix TDS mass flow
    m1.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].unfix()

    return m1


m1 = build_seawater_prop_model()


# =======================================================================================
# Build the model for NaCl property package
def build_NaCl_prop_model():
    m2 = ConcreteModel()
    m2.fs = FlowsheetBlock(dynamic=False)
    m2.fs.properties = NaClParameterBlock()
    m2.fs.feed = Feed(property_package=m2.fs.properties)
    m2.fs.feed.properties[0].pressure_osm_phase
    m2.fs.feed.properties[0].conc_mass_phase_comp
    m2.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
    m2.fs.feed.properties[0].flow_mass_phase_comp["Liq", "NaCl"].fix(0.035)
    m2.fs.feed.properties[0].temperature.fix(273 + 25)
    m2.fs.feed.properties[0].pressure.fix(101325)

    print(f"Degrees of freedom: {degrees_of_freedom(m2)}\n")

    # Initialize and solve for the initial conditions
    m2.fs.feed.initialize()
    results = solver.solve(m2)
    print(f"Solve termination {results.solver.termination_condition}")
    # Unfix NaCl mass flow
    m2.fs.feed.properties[0].flow_mass_phase_comp["Liq", "NaCl"].unfix()

    return m2


m2 = build_NaCl_prop_model()

# =======================================================================================
# Define a range of concentrations
low_concs = np.linspace(5, 70, 50)  # g/L
# Create empty lists to store osmotic pressures for each model
p_osm1_low = []
p_osm2_low = []


for c in low_concs:
    m1.fs.feed.properties[0].conc_mass_phase_comp["Liq", "TDS"].fix(c)
    m2.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"].fix(c)
    results1 = solver.solve(m1)
    results2 = solver.solve(m2)
    p_osm1_low.append(
        value(
            pyunits.convert(
                m1.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )
    p_osm2_low.append(
        value(
            pyunits.convert(
                m2.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )


# =======================================================================================
# Plot the results

fig, ax = plt.subplots()

ax.scatter(low_concs, p_osm1_low, color="blue", label="Seawater Property Package")
ax.scatter(low_concs, p_osm2_low, color="red", label="NaCl Property Package")
ax.set_xlabel("Concentration (g/L)")
ax.set_ylabel("Osmotic Pressure (bar)")
ax.set_title("Osmotic Pressure vs Concentration (Low Range)")
ax.legend()

In [ ]:
m1 = build_seawater_prop_model()
m2 = build_NaCl_prop_model()

# =======================================================================================
# Define a range of concentrations
high_concs = np.linspace(80, 250, 50)  # g/L
# Create empty lists to store osmotic pressures for each model
p_osm1_high = []
p_osm2_high = []

for c in high_concs:
    m1.fs.feed.properties[0].conc_mass_phase_comp["Liq", "TDS"].fix(c)
    m2.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"].fix(c)
    results1 = solver.solve(m1)
    results2 = solver.solve(m2)
    p_osm1_high.append(
        value(
            pyunits.convert(
                m1.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )
    p_osm2_high.append(
        value(
            pyunits.convert(
                m2.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )

# =======================================================================================
# Plot the results

fig, ax = plt.subplots()

ax.scatter(high_concs, p_osm1_high, color="blue", label="Seawater Property Package")
ax.scatter(high_concs, p_osm2_high, color="red", label="NaCl Property Package")
ax.set_xlabel("Concentration (g/L)")
ax.set_ylabel("Osmotic Pressure (bar)")
ax.set_title("Osmotic Pressure vs Concentration (High Range)")
ax.legend()